In [1]:
import abc

from openff.toolkit import unit
from openff.toolkit.typing.engines.smirnoff.parameters import (
    ElectrostaticsHandler,
    IncompatibleParameterError,
    LibraryChargeHandler,
    ParameterAttribute,
    ParameterHandler,
    ParameterType,
    ToolkitAM1BCCHandler,
    VirtualSiteHandler,
    _allow_only,
    vdWHandler,
    ImproperTorsionHandler,
    BondHandler
)


class _CustomBondedHandler(ParameterHandler, abc.ABC):
    """Base class for custom bonded parameter handlers (bonds, angles, torsions)."""

    periodicity = ParameterAttribute(
        default=1, converter=int
    )

    def check_handler_compatibility(self, other_handler: ParameterHandler):
        """Checks whether this ParameterHandler encodes compatible physics as another
        ParameterHandler. This is called if a second handler is attempted to be
        initialized for the same tag.
        
        Parameters
        ----------
        other_handler : a ParameterHandler object
            The handler to compare to.
        
        Raises
        ------
        IncompatibleParameterError if handler_kwargs are incompatible with existing
        parameters.
        """

        if self.__class__ != other_handler.__class__:
            raise IncompatibleParameterError(
                f"{self.__class__.__name__} and {other_handler.__class__.__name__} are not compatible."
            )

        int_attrs_to_compare = ["periodicity"]

        self._check_attributes_are_equal(
            other_handler,
            identical_attrs=int_attrs_to_compare,
            tolerance_attrs=[],
            tolerance=self._SCALETOL,
        )



class HarmonicHeightHandler(_CustomBondedHandler):

    class HarmonicHeightType(ParameterType):
        """Defines parameters for harmonic height restraint."""

        _VALENCE_TYPE = "ImproperTorsion"
        _ELEMENT_NAME = "ImproperTorsion"

        k = ParameterAttribute(
            default=None, unit=unit.kilojoule_per_mole / unit.nanometer**2
        )
        h0 = ParameterAttribute(
            default=None, unit=unit.nanometer
        )

    _TAGNAME = "HarmonicHeight"
    _INFOTYPE = HarmonicHeightType

    def create_force(self, system, topology):
        """Applies the harmonic height potential to the OpenMM system."""
        
        force = CustomCompoundBondForce(4, 
            "0.5 * k * (h - h0)^2;"
            "h = abs((Nx*(x2-x1) + Ny*(y2-y1) + Nz*(z2-z1)) / sqrt(Nx^2 + Ny^2 + Nz^2));"
            "Nx = (y3-y1)*(z4-z1) - (z3-z1)*(y4-y1);"
            "Ny = (z3-z1)*(x4-x1) - (x3-x1)*(z4-z1);"
            "Nz = (x3-x1)*(y4-y1) - (y3-y1)*(x4-x1);"
        )
        force.addPerBondParameter("k")
        force.addPerBondParameter("h0")

        for parameter in self.parameters:
            smirks = parameter.smirks
            k = parameter.k
            h0 = parameter.h0

            for match in topology.chemical_environment_matches(smirks):
                atom_indices = tuple(match) 

                force.addBond(
                    atom_indices,
                    [k.m_as(unit.kilojoule_per_mole / unit.nanometer**2),
                     h0.m_as(unit.nanometer)]
                )

        system.addForce(force)




In [2]:
from openff.toolkit import ForceField, Molecule

force_field = ForceField(load_plugins=True)

harmonic_height_handler = force_field.get_parameter_handler("HarmonicHeight")

harmonic_height_handler.add_parameter(
    {
        "smirks": "[*:1]~[*:2](~[*:3])~[*:4]", 
        "k": 10.0 * unit.kilojoule_per_mole / unit.nanometer ** 2,  
        "h0": 0.1 * unit.nanometer,  
    }
)

harmonic_height_handler.add_parameter(
    {
        "smirks": "[#6X3:1]~[#7:2](~[#6X3:3])~[#1:4]",
        "k": 50.0 * unit.kilojoule_per_mole / unit.nanometer ** 2,  
        "h0": 0.05 * unit.nanometer,  
    }
)


In [3]:
# force_field.register_parameter_handler(harmonic_height_handler)

topology = Molecule.from_mapped_smiles("[H:2][O:1][H:3]").to_topology()

matches = force_field.label_molecules(topology)

# the matches of the 0th molecule in the handler tagged "Buckingham"
# printed as key-val pairs of atom indices and parameter types
print(*matches[0]["HarmonicHeight"].items())

In [4]:
import math
from typing import Dict, Iterable, Literal, Set, Tuple, Type, TypeVar, Union

from openff.interchange import Interchange
from openff.interchange.components.potentials import Potential
from openff.interchange.exceptions import InvalidParameterHandlerError
from openff.interchange.models import VirtualSiteKey
from openff.interchange.smirnoff._base import SMIRNOFFCollection
from openff.interchange.smirnoff._nonbonded import (
    SMIRNOFFvdWCollection,
    _SMIRNOFFNonbondedCollection,
)
from openff.models.types import FloatQuantity
from openff.toolkit import Quantity, Topology, unit
from openff.toolkit.topology import Atom
from openff.toolkit.typing.engines.smirnoff.parameters import ParameterHandler
from openmm import CustomManyParticleForce, openmm

from smirnoff_plugins.handlers.bonded import (
    HarmonicHeightHandler
)

T = TypeVar("T", bound="_BondedPlugin")


class SMIRNOFFImproperTorsionCollection(SMIRNOFFCollection):
    """Handles improper torsions in the SMIRNOFF force field."""

    is_plugin: bool = True
    acts_as: str = "improper-torsion"

    def store_potentials(self, parameter_handler):
        """Store improper torsion parameters from the parameter handler."""
        for potential_key in self.key_map.values():
            smirks = potential_key.id
            parameter = parameter_handler.parameters[smirks]

            self.potentials[potential_key] = Potential(
                parameters={
                    "k": parameter.k,
                    "periodicity": parameter.periodicity,
                    "phase": parameter.phase,
                },
            )

    @classmethod
    def potential_parameters(cls):
        return ("k", "periodicity", "phase")

    @classmethod
    def supported_parameters(cls):
        return "smirks", "id", "k", "periodicity", "phase"

    @classmethod
    def allowed_parameter_handlers(cls):
        return (HarmonicHeightHandler,)

    def modify_openmm_forces(self, interchange, system, *args):
        """Applies the improper torsion potential to an OpenMM system."""
        force = CustomTorsionForce("0.5 * k * (1 + cos(periodicity * theta - phase))")
        force.addPerTorsionParameter("k")
        force.addPerTorsionParameter("periodicity")
        force.addPerTorsionParameter("phase")

        for key, val in self.key_map.items():
            atom_indices = key.atom_indices

            force.addTorsion(
                atom_indices[0], atom_indices[1], atom_indices[2], atom_indices[3],
                [
                    self.potentials[val].parameters["k"].m_as("kilojoule_per_mole"),
                    self.potentials[val].parameters["periodicity"],
                    self.potentials[val].parameters["phase"].m_as("degree"),
                ],
            )

        system.addForce(force)


class SMIRNOFFAxilrodTellerCollection(SMIRNOFFCollection):
    """
    Standard Axilrod-Teller potential from <https://aip.scitation.org/doi/10.1063/1.1723844>.
    """

    expression: str = (
        "0.5 * k * (h - h0)^2;"
        "h = abs((Nx*(x2-x1) + Ny*(y2-y1) + Nz*(z2-z1)) / sqrt(Nx^2 + Ny^2 + Nz^2));"
        "Nx = (y3-y1)*(z4-z1) - (z3-z1)*(y4-y1);"
        "Ny = (z3-z1)*(x4-x1) - (x3-x1)*(z4-z1);"
        "Nz = (x3-x1)*(y4-y1) - (y3-y1)*(x4-x1);"
    )


    type: Literal["HarmonicHeight"] = "HarmonicHeight"

    is_plugin: bool = True
    acts_as: str = ""
    periodic_method: str = "cutoff-periodic"
    nonperiodic_method: str = "cutoff-nonperiodic"
    cutoff: unit.Quantity = unit.Quantity(0.9, unit.nanometer)

    def store_potentials(self, parameter_handler):
        """Store height restraint parameters from the parameter handler."""
        self.nonperiodic_method = parameter_handler.nonperiodic_method
        self.periodic_method = parameter_handler.periodic_method
        self.cutoff = parameter_handler.cutoff

        for potential_key in self.key_map.values():
            smirks = potential_key.id
            parameter = parameter_handler.parameters[smirks]

            self.potentials[potential_key] = Potential(
                parameters={"k": parameter.k, "h0": parameter.h0},
            )

    @classmethod
    def potential_parameters(cls):
        return ("k", "h0")

    @classmethod
    def supported_parameters(cls):
        return "smirks", "id", "k", "h0"

    @classmethod
    def allowed_parameter_handlers(cls):
        return (HarmonicHeightHandler,)

    def modify_openmm_forces(
        self,
        interchange,
        system,
        add_constrained_forces: bool,
        constrained_pairs: Set[Tuple[int, ...]],
        particle_map: Dict[Union[int, "VirtualSiteKey"], int],
    ):
        """Applies the harmonic height potential to an OpenMM system."""
        force = CustomCompoundBondForce(4, self.expression)
        force.addPerBondParameter("k")
        force.addPerBondParameter("h0")

        topology = interchange.topology

        for key, val in self.key_map.items():
            atom_indices = key.atom_indices

            force.addBond(
                atom_indices,
                [
                    self.potentials[val].parameters["k"].m_as("kilojoule_per_mole / nanometer**2"),
                    self.potentials[val].parameters["h0"].m_as("nanometer"),
                ],
            )

        system.addForce(force)

    def modify_parameters(
        self,
        original_parameters: Dict[str, unit.Quantity],
    ) -> Dict[str, float]:
        """Converts the parameters to OpenMM-compatible units."""
        _units = {"k": unit.kilojoule_per_mole / unit.nanometer**2, "h0": unit.nanometer}

        return {
            "k": original_parameters["k"].m_as(_units["k"]),
            "h0": original_parameters["h0"].m_as(_units["h0"]),
        }


In [5]:
from openff.interchange.smirnoff._create import (
    _SUPPORTED_PARAMETER_HANDLERS,
    _PLUGIN_CLASS_MAPPING,
)


_SUPPORTED_PARAMETER_HANDLERS.add("HarmonicHeight")
_PLUGIN_CLASS_MAPPING[HarmonicHeightHandler] = SMIRNOFFAxilrodTellerCollection


In [6]:
from openff.interchange import Interchange

Interchange.from_smirnoff(force_field, topology)

InvalidParameterHandlerError: <class 'smirnoff_plugins.handlers.bonded.ImproperTorsionHandler'>

https://docs.openforcefield.org/projects/interchange/en/stable/using/plugins.html#examples